## Example of the `aitlas` toolbox in the context of image segmentation


### Importing required packages

In [1]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
from aitlas.datasets import TiiLIDARDatasetObjectDetection
from aitlas.models import FasterRCNN
model_config = TiiLIDARDatasetObjectDetection.get_fixed_model_config()

ImportError: cannot import name 'TiiLIDARDatasetObjectDetection' from 'aitlas.datasets' (C:\Users\ncoz\AppData\Local\miniconda3\envs\adaf2\lib\site-packages\aitlas\datasets\__init__.py)

### Loading train, validation and test data

Input parameters for train, validation and test data:

- **batch_size**: The number of samples processed before the model is updated. A larger batch size can speed up processing but requires more memory.
- **num_workers**: The number of worker processes that will be used for processing data. Increasing the number of workers can significantly speed up data processing, however, it also increases memory and CPU/GPU usage.
- **object_class**: A parameter that specifies the type of archaeological object you are interested in processing, e.g., 'AO', 'barrow', 'enclosure', 'ringfort'.
- **vizuelization_type**: The vizuelization type used for the patches, e.g., 'SLRM'.
- **DFM_quality**: List of annotation qualities to be included in the processed data, e.g., '1,2'.
- **shuffle**: Determines whether the data should be shuffled before being processed. 
- **data_dir**: The directory path where the input data is stored. 
- **annotations_dir**: The directory path where the labels (i.e., bounding boxes) are stored. 
- **transforms**: A list of transformations applied to the input data during processing.
- **target_transforms**: A list of transformations applied to the segmentation masks during processing.
- **joint_transforms**: Transformations applied simultaneously to both the input data and segmentation masks.

In [ ]:
batch_size = 16
num_workers = 4
object_class = "ringfort"
vizuelization_type = "SLRM"

In [ ]:
train_dataset_config = {
    "batch_size": batch_size,
    "num_workers": num_workers,
    "object_class": object_class,
    "DFM_quality": "1,2",
    "vizuelization_type": vizuelization_type,
    "shuffle": True,
    "data_dir": "/Users/anakostovska/Dropbox/aitlas_v1/retrain_model/demo_data/samples/train",
    "annotations_dir": "/Users/anakostovska/Dropbox/aitlas_v1/retrain_model/demo_data/labels/object_detection_bbox/train",
    "joint_transforms": ["aitlas.transforms.TransformFlipHVToTensorV2"]
}
train_dataset = TiiLIDARDatasetObjectDetection(train_dataset_config)

validation_dataset_config = {
    "batch_size": batch_size,
    "num_workers": num_workers,
    "object_class": object_class,
    "DFM_quality": "1,2",
    "vizuelization_type": vizuelization_type,
    "shuffle": False,
    "data_dir": "/Users/anakostovska/Dropbox/aitlas_v1/retrain_model/demo_data/samples/validation",
    "annotations_dir": "/Users/anakostovska/Dropbox/aitlas_v1/retrain_model/demo_data/labels/object_detection_bbox/validation",
    "joint_transforms": ["aitlas.transforms.TransformToTensorV2"],
}
validation_dataset = TiiLIDARDatasetObjectDetection(validation_dataset_config)

test_dataset_config = {
    "batch_size": batch_size,
    "num_workers": num_workers,
    "object_class": object_class,
    "DFM_quality": "1,2",
    "vizuelization_type": vizuelization_type,
    "shuffle": False,
    "data_dir": "/Users/anakostovska/Dropbox/aitlas_v1/retrain_model/demo_data/samples/test",
    "annotations_dir": "/Users/anakostovska/Dropbox/aitlas_v1/retrain_model/demo_data/labels/object_detection_bbox/test",
    "joint_transforms": ["aitlas.transforms.TransformToTensorV2"],
}
test_dataset = TiiLIDARDatasetObjectDetection(test_dataset_config)


len(train_dataset), len(validation_dataset), len(test_dataset)

### Model creation

In [ ]:
model = FasterRCNN(model_config)
model.prepare()

### Loading pretrained ADAF model (optional)

If you don't want to use an existing model, you can skip this step. If you do want to use one, set the model path, uncomment the lines, and run the cell to load the model into memory.

In [ ]:
model_path = "/Users/anakostovska/Dropbox/aitlas_v1/inference/data/models/object_detection/OD_ringfort.tar"
model.load_model(model_path)

### Training the model

Input parameters: 
- **epochs**: The total number of training cycles the model will undergo. Each epoch represents one complete pass of the training dataset through the model.
- **model_directory**: Path to the directory where the trained model and its checkpoints will be saved. This is used for storing the model during and after training.
- **run_id**: Name of the subdirectory within the model_directory to store results from different runs

In [ ]:
epochs = 20
model_directory = "./models/object_detection/"
run_id = 'ringfort_1_2'

In [ ]:
model.train_and_evaluate_model(
    train_dataset=train_dataset,
    val_dataset=validation_dataset,
    epochs=epochs,
    model_directory=model_directory,
    run_id=run_id
);

### Model evaluation

In [ ]:
model = FasterRCNN(model_config)
model.prepare()
model.running_metrics.reset()
model_path = "/Users/anakostovska/Dropbox/aitlas_v1/retrain_model/models/object_detection/ringfort_1_2/best_checkpoint_1710347562_1.pth.tar" # update the path!
model.evaluate(dataset=test_dataset, model_path=model_path)
model.running_metrics.get_scores(model.metrics)